# Better than the First
## IMDb Web Scraping
The following code are scrapes the IMDb pages to get the movies details such as the film series name, film movie name, year of release, budget, worldwide gross, movie production house, movie poster link and the others.

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import bs4
import requests
import re
import time
from sqlalchemy import create_engine

### First 10,000 Movies

In [2]:
proxies = {'http' : 'http://206.189.157.23:80',
           'https' : 'https://206.189.157.23:80',}
imdb_pages = [""] + ["&start=" + str(i) for i in range(51,10000,50)]

for count in imdb_pages:
    imdb_list = (requests.get("https://www.imdb.com/search/title/?title_type"
                             "=movie&sort=boxoffice_gross_us,desc" + count + 
                             "&explore=title_type,genres", 
                             proxies = proxies).text)
    imdb_list_soup = bs4.BeautifulSoup(imdb_list)
    imdb_list_result = []
    for imdb_line_item in imdb_list_soup.select('''div.lister > .lister-list >
                                .lister-item > .lister-item-content'''):
        imdb_movie = imdb_line_item.h3.a.text
        imdb_link = "https://www.imdb.com" + imdb_line_item.h3.a['href']
        imdb_image = (imdb_line_item.parent
                    .find(class_= 'lister-item-image float-left').img['src'])
        try:
            imdb_genre = (imdb_line_item.p.find(class_= 'genre')
                          .text.replace(",","").split()) 
            imdb_runtime = imdb_line_item.p.find(class_= 'runtime').text 
            imdb_certificate = (imdb_line_item.p.find(class_= 'certificate')
                                .text) 
        except:
            imdb_genre, imdb_runtime, imdb_certificate = "", "", ""
        try:
            imdb_year = (re.findall(r'\d{4}', 
                            str(imdb_line_item.h3
                            .find('span', {'class':'lister-item-year'})))[0])
        except:
            imdb_year = ""
        try:
            imdb_stars = (imdb_line_item
                          .find(class_= 'ratings-bar').strong.text)
            imdb_metascore = (imdb_line_item
                              .find(class_= 'inline-block ratings-metascore')
                              .span.text)
        except:
            imdb_stars, imdb_metascore= "", ""   
        try:
            imdb_votes = (imdb_line_item
                          .find(class_= 'sort-num_votes-visible')
                          .find_all('span', {'name':'nv'})[0].text)
            imdb_gross = (imdb_line_item
                          .find(class_= 'sort-num_votes-visible')
                          .find_all('span', {'name':'nv'})[1].text)
        except:
            imdb_votes, imdb_gross = "", ""
        imdb_list_result.append([imdb_movie, imdb_year, " ".join(imdb_genre), 
                                 imdb_runtime, imdb_certificate, imdb_stars, 
                                 imdb_metascore, imdb_votes, imdb_gross, 
                                 imdb_link, imdb_image])

    df_imdb_list = pd.DataFrame(imdb_list_result, columns=["Movie", "Year", 
                                "Genre", "Runtime", "Certificate", 
                                "Ratings Star",  "Metascore", "Votes", 
                                "Gross", "Movie Link", "Poster Link"])   
    df_imdb_list.to_sql('IMDb_movies', 'sqlite:////mnt/processed/private/'
                        'msds2021/slt10c/IMDb_movies.db', if_exists='append')
    time.sleep(np.random.random())

### Beyond 10,000 Movies

In [3]:
imdb_next_page = ("https://www.imdb.com/search/title/?title_type=movie&sort="
                 "boxoffice_gross_us,desc&start=9951&explore=title_type,"
                 "genres&ref_=adv_nxt")
        
for count in range(1,200):
    imdb_list = requests.get(imdb_next_page, 
                          proxies = proxies).text
    imdb_list_soup = bs4.BeautifulSoup(imdb_list)
    imdb_list_result = []
    imdb_next_page = ("https://www.imdb.com" + imdb_list_soup
                      .find('a', {'class':'lister-page-next'})['href'])
        
    for imdb_line_item in imdb_list_soup.select('''div.lister > .lister-list >
                                    .lister-item > .lister-item-content'''):
        try: 
            imdb_movie = imdb_line_item.h3.a.text
            imdb_link = "https://www.imdb.com" + imdb_line_item.h3.a['href']
            imdb_image = (imdb_line_item.parent
                          .find(class_= 'lister-item-image float-left')
                          .img['src'])
        except:
            imdb_movie, imdb_link, imdb_image = "", "", ""
        try:
            imdb_genre = (imdb_line_item.p.find(class_= 'genre')
                          .text.replace(",","").split()) 
            imdb_runtime = imdb_line_item.p.find(class_= 'runtime').text 
            imdb_certificate = (imdb_line_item.p
                                .find(class_= 'certificate').text) 
        except:
            imdb_genre, imdb_runtime, imdb_certificate = "", "", ""
        try:
            imdb_year = (re.findall(r'\d{4}', str(imdb_line_item.h3
                          .find('span', {'class':'lister-item-year'})))[0])
        except:
            imdb_year = ""
        try:
            imdb_stars = (imdb_line_item
                          .find(class_= 'ratings-bar').strong.text)
            imdb_metascore = (imdb_line_item
                              .find(class_= 'inline-block ratings-metascore')
                              .span.text)
        except:
            imdb_stars, imdb_metascore= "", ""   
        try:
            imdb_votes = (imdb_line_item
                          .find(class_= 'sort-num_votes-visible')
                          .find_all('span', {'name':'nv'})[0].text)
            imdb_gross = (imdb_line_item
                          .find(class_= 'sort-num_votes-visible')
                          .find_all('span', {'name':'nv'})[1].text)
        except:
            imdb_votes, imdb_gross = "", ""
        imdb_list_result.append([imdb_movie, imdb_year, " ".join(imdb_genre), 
                                 imdb_runtime, imdb_certificate, imdb_stars, 
                                 imdb_metascore, imdb_votes, imdb_gross, 
                                 imdb_link, imdb_image])

    df_imdb_list = pd.DataFrame(imdb_list_result, columns=["Movie", "Year", 
                                        "Genre", "Runtime", "Certificate", 
                                        "Ratings Star",  "Metascore", "Votes",
                                        "Gross", "Movie Link", "Poster Link"])   
    df_imdb_list.to_sql('IMDb_movies', 'sqlite:////mnt/processed/private/'
                        'msds2021/slt10c/IMDb_movies.db', if_exists='append')
    time.sleep(np.random.random())

## Per Page Web Scraping

In [4]:
conn = sqlite3.connect('/mnt/processed/private/msds2021/'
                       'slt10c/IMDb_movies.db')
conn.execute('SELECT * FROM IMDb_movies')

In [5]:
df = pd.read_sql("""SELECT * FROM IMDb_movies""", conn)
df

,index,Movie,Year,Genre,Runtime,Certificate,Ratings Star,Metascore,Votes,Gross,Movie Link,Poster Link
0,0,Star Wars: Episode VII - The Force Awakens,2015,Action Adventure Sci-Fi,138 min,PG,7.9,80,"840,172",$936.66M,https://www.imdb.com/title/tt2488496/,https://m.media-amazon.com/images/G/01/imdb/im...
1,1,Avengers: Endgame,2019,Action Adventure Drama,181 min,PG13,8.4,78,"730,971",$858.37M,https://www.imdb.com/title/tt4154796/,https://m.media-amazon.com/images/G/01/imdb/im...
2,2,Avatar,2009,Action Adventure Fantasy,162 min,PG,7.8,83,"1,094,980",$760.51M,https://www.imdb.com/title/tt0499549/,https://m.media-amazon.com/images/G/01/imdb/im...
3,3,Black Panther,2018,Action Adventure Sci-Fi,134 min,PG,7.3,88,"585,371",$700.06M,https://www.imdb.com/title/tt1825683/,https://m.media-amazon.com/images/G/01/imdb/im...
4,4,Avengers: Infinity War,2018,Action Adventure Sci-Fi,149 min,PG13,8.4,68,"781,829",$678.82M,https://www.imdb.com/title/tt4154756/,https://m.media-amazon.com/images/G/01/imdb/im...
...,...,...,...,...,...,...,...,...,...,...,...,...
19945,45,Bichagada Majaka,,,,,,,,,https://www.imdb.com/title/tt9761284/,https://m.media-amazon.com/images/G/01/imdb/im...
19946,46,Gamer,,,,,,,,,https://www.imdb.com/title/tt9761276/,https://m.media-amazon.com/images/G/01/imdb/im...
19947,47,Pei Ellam Paavam,2019,,,,,,,,https://www.imdb.com/title/tt9761272/,https://m.media-amazon.com/images/G/01/imdb/im...
19948,48,Uranga Puli,2019,,,,,,,,https://www.imdb.com/title/tt9761270/,https://m.media-amazon.com/images/G/01/imdb/im...


In [6]:
imdb_pages = df['Movie Link']
for count in imdb_pages:
    imdb_list = requests.get(count, proxies = proxies).text
    imdb_list_soup = bs4.BeautifulSoup(imdb_list)
    imdb_list_result = []
    imdb_country, imdb_language, imdb_budget = "", "", ""
    imdb_worldgross, imdb_company = "", ""
    for imdb_line_item in imdb_list_soup.select('div.article#titleDetails'):
        for details in imdb_line_item.find_all("h4"):
            if details.text == "Country:":
                imdb_country = details.parent.a.text
            if details.text == "Language:":
                imdb_language = details.parent.a.text
            if details.text == "Budget:":
                imdb_budget = (''.join(details.parent
                                       .find_all(text=True, recursive=False))
                                       .strip())  
            if details.text == "Cumulative Worldwide Gross:":
                imdb_worldgross = (''.join(details.parent
                                       .find_all(text=True, recursive=False))
                                       .strip())
            if details.text == "Production Co:":
                imdb_company = (", ".join([x.get_text().strip() for x in 
                                           details.parent.find_all("a")]))
    try: 
        imdb_poster = (imdb_list_soup
                       .find(name='div', class_='poster').img['src'])
    except:
        imdb_poster =  ""
    imdb_list_result.append([count, imdb_country, imdb_language, imdb_budget, 
                             imdb_worldgross, imdb_company, imdb_poster])
    df_imdb_list = pd.DataFrame(imdb_list_result, columns=["Movie Link", 
                                  "Country", "Language", "Budget", 
                                  "Worldwide Gross", "Production Co", 
                                  "Poster"])  
    df_imdb_list.to_sql('IMDb_movies_perpage', 
                        'sqlite:////mnt/processed/private/msds2021/'
                        'slt10c/IMDb_movies_perpage.db', if_exists='append')
    time.sleep(np.random.random())

In [7]:
conn2 = sqlite3.connect('/mnt/processed/private/msds2021/slt10c/'
                        'IMDb_movies_perpage.db')
conn2.execute('SELECT * FROM IMDb_movies_perpage')

In [8]:
df_2 = pd.read_sql("""SELECT * FROM IMDb_movies_perpage""", conn2)
df_2

,index,Movie Link,Country,Language,Budget,Worldwide Gross,Production Co,Poster
0,0,https://www.imdb.com/title/tt2488496/,USA,English,"$245,000,000","$2,068,223,624","Lucasfilm, Bad Robot, Truenorth Productions, S...",https://m.media-amazon.com/images/M/MV5BOTAzOD...
1,0,https://www.imdb.com/title/tt4154796/,USA,English,"$356,000,000","$2,797,800,564","Marvel Studios, Walt Disney Pictures, See more",https://m.media-amazon.com/images/M/MV5BMTc5MD...
2,0,https://www.imdb.com/title/tt0499549/,USA,English,"$237,000,000","$2,790,439,000","Twentieth Century Fox, Dune Entertainment, Lig...",https://m.media-amazon.com/images/M/MV5BMTYwOT...
3,0,https://www.imdb.com/title/tt1825683/,USA,English,"$200,000,000","$1,346,913,161","Marvel Studios, Walt Disney Pictures, See more",https://m.media-amazon.com/images/M/MV5BMTg1MT...
4,0,https://www.imdb.com/title/tt4154756/,USA,English,"$321,000,000","$2,048,359,754","Marvel Studios, Jason Roberts Productions, Sou...",https://m.media-amazon.com/images/M/MV5BMjMxNj...
...,...,...,...,...,...,...,...,...
19945,0,https://www.imdb.com/title/tt9761284/,India,Telugu,,,,https://m.media-amazon.com/images/M/MV5BNzY1MD...
19946,0,https://www.imdb.com/title/tt9761276/,India,Telugu,,,,
19947,0,https://www.imdb.com/title/tt9761272/,India,Tamil,,,"Tharakan Cinemas, See more",https://m.media-amazon.com/images/M/MV5BMjIyNj...
19948,0,https://www.imdb.com/title/tt9761270/,India,Tamil,,,"CJ Pictures, See more",https://m.media-amazon.com/images/M/MV5BNTFlMj...
